# Исследование надежности заемщиков

## Описание проекта
**Цель проекта** - проанализировать данные банка, чтобы определить, влияет ли семейное положение, количество детей и другие факторы на погашение кредита в срок. Данные содержат статистику о платежеспособности клиентов по кредитам.

**План проекта**:
- загрузить и изучить данные
- выполнить предобработку данных
- исследовать данные и ответить на вопросы: 
 - есть ли зависимость между количеством детей и возвратом кредита в срок?
 - есть ли зависимость между семейным положением и возвратом кредита в срок?
 - есть ли зависимость между уровнем дохода и возвратом кредита в срок?
 - как разные цели кредита влияют на его возврат в срок?
- составить общий вывод

## Загрузим и изучим данные

In [1]:
import pandas as pd

try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

In [3]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

Выведем количество пропущенных значений для каждого столбца

In [5]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

В двух столбцах есть пропущенные значения. Один из них — days_employed. Пропуски в этом столбце мы обработаем на следующем этапе. Другой столбец с пропущенными значениями — total_income — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце нужно медианным значением по каждому типу из столбца income_type.

In [6]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

В данных могут встречаться артефакты (аномалии) — значения, которые не отражают действительность и появились по какой-то ошибке. таким артефактом будет отрицательное количество дней трудового стажа в столбце `days_employed`. Для реальных данных это нормально. Обработаем значения в этом столбце: заменим все отрицательные значения положительными.

In [8]:
data['days_employed'] = data['days_employed'].abs()

Для каждого типа занятости выведем медианное значение трудового стажа `days_employed` в днях

In [9]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставим их как есть.

Выведем перечень уникальных значений столбца `children`

In [10]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

В столбце children есть два аномальных значения. Удалим строки, в которых встречаются такие аномальные значения из датафрейма data.

In [12]:
data = data[(data['children'] != -1) & (data['children'] != 20)]
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

Заполним пропуски в столбце `days_employed` медианными значениями по каждого типа занятости `income_type`

In [13]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

Убедимся, что все пропуски заполнены.

In [14]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

Заменим вещественный тип данных в столбце total_income на целочисленный

In [16]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов


Обработаем неявные дубликаты в столбце education. В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведём их к нижнему регистру.

In [17]:
data['education'] = data['education'].str.lower()

Выведем количество строк-дубликатов в данных. Если такие строки присутствуют, удалим их

In [18]:
data.duplicated().sum()

71

In [19]:
data = data.drop_duplicates()

### Категоризация данных

На основании диапазонов, создадим в датафрейме `data` столбец `total_income_category` с категориями:

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.

In [20]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [21]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

Выведем на экран перечень уникальных целей взятия кредита из столбца `purpose`

In [22]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Создадим функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.


In [23]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [25]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы


## Исследуем данные и ответим на вопросы

### Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [27]:
children_debt = data.groupby('children')['debt'].agg(['count', 'sum', 'mean']) 
children_debt

,count,sum,mean
children,,,
0,14091,1063,0.075438
1,4808,444,0.092346
2,2052,194,0.094542
3,330,27,0.081818
4,41,4,0.097561
5,9,0,0.000000


**Вывод:** Исследование показывает, что среди самых больших групп, а это с 0, 1 и 2мя детьми прослеживается закономерность роста колличества просрочек с ростом колличества детей. Остальные группы - 3, 4 и 5 детей не репрезентативны ввиду маленькой выборки. 


### Есть ли зависимость между семейным положением и возвратом кредита в срок?


In [29]:
family_debt = data.groupby('family_status')['debt'].agg(['count', 'sum', 'mean']) 
family_debt


,count,sum,mean
family_status,,,
Не женат / не замужем,2796,273,0.097639
в разводе,1189,84,0.070648
вдовец / вдова,951,63,0.066246
гражданский брак,4134,385,0.093130
женат / замужем,12261,927,0.075606


**Вывод:** Если прослеживать закономерность между самыми большими группами, то мы видим, что люди, находящиеся в официальном браке, реже допускают просрочки (на ~2%) , чем не женатые или не замужние и состоящие в гражданском браке. 

### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [31]:
income_debt = data.groupby('total_income_category')['debt'].agg(['count', 'sum', 'mean']) 
income_debt

,count,sum,mean
total_income_category,,,
A,25,2,0.080000
B,5014,354,0.070602
C,15921,1353,0.084982
D,349,21,0.060172
E,22,2,0.090909


**Вывод:** Делая вывод по двум самым крупным категориям по уровню дохода, а именно: B (200001–1000000) и С (50001-200000), мы видим, что клиенты с большим доходом реже (на ~1,5%) допускали просрочки. С уменьшением дохода растёт частота просрочек.

### Как разные цели кредита влияют на его возврат в срок?

In [33]:
purpose_debt = data.groupby('purpose_category')['debt'].agg(['count', 'sum', 'mean']) 
purpose_debt

,count,sum,mean
purpose_category,,,
операции с автомобилем,4279,400,0.093480
операции с недвижимостью,10751,780,0.072551
получение образования,3988,369,0.092528
проведение свадьбы,2313,183,0.079118


**Вывод:** Клиенты, берущие кредит для операций с недвижимостью и для проведения свадьбы реже допускали просрочки (разница ~2%). Кредиты для операций с автомобилем и получения образования - наиболее рискованные категории. 

## Общий вывод.

При предобработке данных были обнаружены пропуски в колонках с колличеством отработанных дней и суммарным доходом. Пропуски были заполненны средним медианным значением по группам, в зависимости от типа занятости (income_type). Были добавленны категории клиентов по суммарному доходу (total_income_category). Выявлены и убраны повторы, явные и неявные. В колонке с колличеством детей убраны аномальные значения. Во время предобработки были потеряны меньше 1% данных, что не скажется на итоговом результате.

Было проведено исследование влияния разных факторов на просрочки по кредитую. Исследование выявило следующие закономерности:
1) Самая низкая вероятность просрочки у клиентов без детей. Далее прослеживается закономерность роста колличества просрочек с ростом колличества детей (1 и 2 ребёнка). 
2) Клиенты, находящиеся в официальном браке, реже допускают просрочки (на ~2%) , чем не женатые или не замужние и состоящие в гражданском браке.
3) С ростом доходов клиента можно заметить снижение вероятности просрочек. Клиенты с доходом от 200 тыс до 1 млн реже (на ~1,5%) допускали просрочки, чем клиенты с доходом от 50 тыс до 200 тыс.). Остальные группы клиентов по уровню доходов были слишком не респрезентативны.
4) Клиенты, берущие кредит для операций с недвижимостью и для проведения свадьбы реже допускали просрочки (разница ~2%). Кредиты для операций с автомобилем и получения образования - наиболее рискованные категории. 

Портрет наиболее надёжного заёмщика выглядит следующим образом: клиент без детей, состоящий в официальном браке, имеющий доход от 200 тыс до 1 млн, берущий кредит для операций с недвижимостью или для проведения свадьбы. 
Портрет наименее надёжного заёмщика: клиент с 1 или 2 детьми, не состоящий в браке или живущий в гражданском браке, с доходом от 50 до 200 тыс, берущий кредит для операций с автомобилем или получения образования. 

Исследование показало, что разница между категориями не столь велика, 1-3 процента. Я бы порекомендовал провести исследование по влиянию совокупности факторов на своевременность выдачи кредита. Или собрать больше данных, чтобы можно было полноценно оценивать все категории заёмщиков.